# Playground

In [1]:
import os
import json
import copy

train_folder = 'data/training'
all_data = []

In [2]:
# Extracting Data from the Training Set
for filename in os.listdir(train_folder): 
    if filename.endswith('.json'):
        with open(os.path.join(train_folder, filename), 'r') as file:
            data = json.load(file)
            train_data = data['train']
            test_data = data['test']
            all_data.append([train_data, test_data])

print("Total Files:", len(os.listdir(train_folder)))
print("Total Data:", len(all_data))

Total Files: 400
Total Data: 400


In [3]:
def data_parser(user_data, hide_test_labels=False, return_answer=False):

    # Making a copy of data
    data = copy.deepcopy(user_data)
    train_samples = []
    test_sample = None
    train, test =  data

    # Parsing Samples
    for sample in train:
        train_samples.append(sample)
    for sample in test:
        answer = sample['output']
        if hide_test_labels:
            sample['output'] = '?'
            test_sample = sample
        else:
            test_sample = sample
            
    if return_answer:
        del data
        return train_samples, test_sample, answer
    else:
        del data
        return train_samples, test_sample

In [4]:
# Importing API Key
from api_keys import together_key, openai_key

In [5]:
# Initializing Langchain LLama Model

from langchain_together import ChatTogether
llm = ChatTogether(api_key=together_key, model="meta-llama/Llama-3.3-70B-Instruct-Turbo")

The idea is to create a Generator Vs Discriminator model, where
- Generator: Its job is to create possible solutions(unique) which can be mapped to the output.
- Discriminator: The job of discriminator is to critique the possible solutions generated by the Generator

In [6]:
# Generator System Prompt

generator_sys_prompt = """
    You are a generator, an assistant tasked with generating possible solutions for a given pattern generation problem.
    Your goal is to produce correct solutions that align with the constraints provided in the problem, leveraging ideas suggested by the Dream AI.

    Problem Description:
    - You will receive:
        1. An array of examples, where each example is a dictionary containing an "input" and "output" pair.
        2. Suggestions from Dream AI in the form of an "ideas" array, which contains 10 simple and unique ideas for solving the problem.
        3. A "solutions" array containing previously generated and validated solutions.
    - Your task is to:
        - Identify the core pattern or principle that maps inputs to outputs using the examples.
        - Incorporate ideas from the "ideas" array to guide and enhance your reasoning process.

    Constraints:
    - Do not replicate solutions already present in the provided "solutions" array.
    - Ensure your solutions are logically valid and adhere to any specific constraints mentioned in the problem.

    Reasoning Framework:
    - Treat this task like a reasoning tree:
        - The root represents the primary goal: to map inputs to outputs effectively.
        - The mid-level branches represent your reasoning process and steps to identify patterns.
        - The leaves represent potential solutions that satisfy the problem constraints.
    - Integrate ideas from the "ideas" array into your reasoning tree to explore novel and creative possibilities.
    - Break down your reasoning step by step to ensure transparency and correctness.

    Guidelines for Solution Generation:
    - Favor symmetry and simplicity; humans tend to appreciate solutions with these qualities.
    - Critically evaluate ideas from Dream AI and select the most promising ones to incorporate into your solution.
    - Verify your reasoning rigorously to ensure alignment with the given examples.
    - For every solution you provide, explain why it works and how ideas influenced your reasoning.

    Output Format:
    - Provide the solution and its reasoning in the following structured format:
        {
            "problem": "Problem description",
            "dream_ideas_used": ["idea_1", "idea_2", ...],  # List the ideas used from the Dream AI array
            "answer": [[...]],  # Your generated solution (e.g., a sequence, array, etc.)
            "reason": "Explanation of why this solution works, including your reasoning process, alignment with the problem constraints, and how Dream AI ideas were incorporated."
        }

    Additional Tips:
    - Approach the task intuitively but verify your assumptions.
    - Use the Dream AI ideas as inspiration, not strict constraints; modify and adapt them as needed.
    - Avoid overcomplicating the reasoning; prioritize clarity and correctness.
    - Continuously critique your reasoning and iterate to improve your solutions.
"""


In [7]:
# Discriminator System Prompt

discriminator_sys_prompt = """
    You are a discriminator, an assistant responsible for critically evaluating solutions generated by another AI model.
    Your primary objective is to verify the correctness, logic, and alignment of the solutions with the problem constraints.
    Additionally, if possible, provide a simple, unique hint that can guide towards better solutions.

    Problem Description:
    - You will be provided with:
        - An array of examples, each consisting of an "input" and "output" pair.
        - A solution generated by another AI model.
        - A list of already tested solutions for reference.

    Evaluation Goals:
    - Ensure the generated solution:
        1. Accurately maps the inputs to the outputs as demonstrated in the given examples.
        2. Does not replicate any solution or reasoning already present in the tested solutions array.

    Testing Methodology:
    1. **Brutal Validation:**
        - Check the solution rigorously against the given examples.
        - If even one example is incorrect or the mapping is inconsistent, reject the solution outright.
        - Confirm that the solution is not a duplicate of any existing solutions in the array.
    2. **Complexity Assessment:**
        - Verify if the reasoning and solution align with human-like simplicity and intuition:
            - Humans prefer simple, easily understandable solutions that follow a top-down approach.
            - If the solution is overly complex or convoluted, it is likely unsuitable as a human-like solution.
        - Ensure the solution’s complexity does not surpass what is necessary to solve the problem effectively.
    3. **Hint Generation (if possible):**
        - If you can identify a unique and simple hint that aligns with the problem constraints and examples:
            - Ensure the hint does not completely overlap with the provided solutions, though slight overlaps are acceptable.
            - Hints should guide towards better reasoning or a possible improvement in the solution while remaining concise.
    4. **Score Assignment:**
        - Assign a score based on the quality of the solution:
            - Higher scores for solutions that are correct, simple, and intuitive.
            - Lower scores for solutions that are overly complex, even if correct.

    Output Format:
    - Provide your evaluation and hint in the following structured format:
    {
        "problem": "Problem description"
        "result": True or False,  # True if the solution is valid, False otherwise
        "score": floating value between 0 and 1,  # Reflecting the quality of the solution
        "reason": "Detailed explanation of why the solution was accepted or rejected, including evaluation of correctness, simplicity, and originality.",
        "hint": "A simple, unique hint to guide towards a better solution, if possible. If no hint is applicable, return null."
    }
    
"""


In [8]:
aggregator_sys_prompt = """
    You are an assistant with exceptional analytical and reasoning skills, tasked with aggregating and refining solutions provided by the generator and discriminator.
    Your primary objective is to synthesize all failed solutions, evaluations, and the provided example set to construct the simplest, most effective, and logically sound final solution.

    Problem Description:
    - The core task is a **pattern recognition problem**, where the goal is to deduce the underlying principles or rules that map inputs to outputs.
    - You will receive:
        1. An array of **examples**, where each example is a dictionary containing:
            - "input": A representation of the input data for a specific case.
            - "output": The corresponding correct output for that input, as defined by the problem.
        2. An array of **solutions**, each containing:
            - A generated solution (candidate mapping or rule) from the generator.
            - Reasoning provided by the generator for why the solution works.
        3. An array of **evaluations**, where each evaluation contains:
            - The discriminator's assessment of a solution (valid/invalid).
            - A score (0 to 1) representing the quality of the solution.
            - A hint (if applicable) to guide improvement.
    - Your task is to aggregate this information to produce a final solution that:
        - Correctly maps all inputs to outputs based on the examples.
        - Is simple, powerful, and adheres to human-like reasoning.
        - Avoids repeating previously failed or invalid approaches.

    Aggregation Methodology:
    1. **Example Alignment:**
        - Use the provided examples to validate and guide your final solution.
        - Ensure that the final solution accurately maps each "input" to the corresponding "output."
        - Look for simple, prominent patterns in the examples that can generalize across all cases.

    2. **Solution Aggregation:**
        - Combine all unique solutions provided by the generator, ensuring no duplicates.
        - Group similar solutions and prioritize those with:
            - The strongest reasoning.
            - High alignment with the examples.
            - Favorable evaluations and scores.
        - Eliminate solutions that fail evaluations or are overly complex.

    3. **Simplistic and Prominent Pattern First:**
        - Identify and solve the most basic and prominent pattern evident from the examples.
        - Prioritize simplicity; the solution should be intuitive and easy to understand while satisfying all constraints.

    4. **Hint Utilization:**
        - Incorporate hints provided by the discriminator to refine or improve the solution.
        - Ensure hints are used to uncover overlooked patterns or simplify the solution, without fully replicating previous solutions.

    5. **Final Solution Construction:**
        - Synthesize the best aspects of valid solutions, evaluations, and hints into a single, cohesive answer.
        - Ensure the final solution is concise, logically valid, and comprehensively addresses the examples and constraints.

    Output Format:
    - Provide the final solution in the following structured format:
    {
        "problem": "Problem description",
        "final_solution": [[...]],  # Your synthesized solution (e.g., a sequence, array, etc.)
        "reason": "Detailed explanation of why this solution is the simplest and most effective, including how it aligns with the example set, evaluations, and hints."
    }

    Additional Guidelines:
    - Always validate the final solution against the examples to ensure correctness.
    - Prioritize simplicity and clarity; avoid overly complex reasoning or solutions.
    - Focus on human-like logical patterns and intuitive problem-solving approaches.
    - Ensure the final solution is unique, avoiding complete overlap with previous solutions.
"""


In [32]:
# Dream System Prompt

dreamstate_sys_prompt = """
    You are a creative assistant tasked with brainstorming and generating innovative ideas to solve a given pattern recognition problem.
    Your primary objective is to produce 10 simple, unique, and logically sound ideas that align with the problem constraints while ensuring they are not duplicates of existing solutions.

    Problem Description:
    - The core task is a **pattern recognition problem**, where the goal is to identify rules or principles that map "input" to "output" based on provided examples.
    - You will receive:
        1. An array of examples, each consisting of:
            - "input": A representation of the input data for a specific case.
            - "output": The corresponding correct output for that input.
        2. A "solutions" array containing already proposed solutions and their reasoning.

    Task Guidelines:
    - Your goal is to generate 10 ideas that:
        - Are simple and intuitive for humans to understand.
        - Solve the pattern recognition problem effectively.
        - Do not completely overlap with solutions already present in the "solutions" array, although slight variations are acceptable.
    - Each idea should be unique and represent a different perspective or approach.

    Idea Generation Methodology:
    1. **Analyze the Examples:**
        - Study the given examples to identify patterns or regularities in how inputs map to outputs.
        - Focus on simplicity and look for the most intuitive relationships.

    2. **Avoid Redundancy:**
        - Cross-check your ideas against the existing solutions array to ensure uniqueness.
        - Modify or refine ideas that overlap significantly with existing solutions.

    3. **Generate Diverse Approaches:**
        - Think of multiple perspectives or strategies to solve the problem.
        - Include ideas based on symmetry, arithmetic patterns, transformations, or any logical rules observed in the examples.

    Output Format:
    - Provide your 10 ideas in the following structured format:
    {
        "problem": "Array of examples shared with you should be passed back",
        "ideas": [
            {
                "idea": "Description of the idea.",
                "reason": "Explanation of why this idea works and how it aligns with the examples."
            },
            ...
        ]
    }

    Additional Tips:
    - Prioritize simplicity and clarity; each idea should be concise and easy to understand.
    - Ensure all ideas are logical and align with the provided examples.
    - If you encounter difficulty generating unique ideas, try abstracting the problem to identify novel perspectives.
"""


In [33]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool, AgentExecutor, initialize_agent
from langchain.chat_models import ChatOpenAI

# Initializing the llm
llm = ChatOpenAI(api_key=openai_key, model="gpt-4o", temperature=0.3)
generator_llm = ChatOpenAI(api_key=openai_key, model="gpt-4o", temperature=0.7)
discriminator_llm = ChatOpenAI(api_key=openai_key, model="gpt-4o", temperature=0.2)
aggregator_llm = ChatOpenAI(api_key=openai_key, model="gpt-4o", temperature=0.5)
dreamstate_llm = ChatOpenAI(api_key=openai_key, model="gpt-4o", temperature=1.0)

In [34]:
# Initializing prompts

dreamstate_prompt = PromptTemplate(
    input_variables=["examples", "solutions_hist"],
    template = """
        {dreamstate_sys_prompt}

        Problem Examples:
        {examples}

        Solutions History: {solutions_hist}
        Generate your solution below:
"""
)

generator_prompt = PromptTemplate(
    input_variables=["examples", "ideas", "solutions_hist"],
    template = """
        {generator_sys_prompt}

        Problem Examples:
        {examples}

        Ideas from Dream AI: {ideas}

        Solutions History:{solutions_hist}
        Generate your solution below:
"""
)

disciminator_prompt = PromptTemplate(
    input_variables=["examples", "generated_solution", "solutions_hist"],
    template = """
        {discriminator_sys_prompt}

        Problem Examples:
        {examples}

        Generated Solution:{generated_solution}
        Solutions History: {solutions_hist}
        Generate your solution below:
"""
)

aggregator_prompt = PromptTemplate(
    input_variables=["examples", "solutions_hist", "evalutation_hist"],
    template = """
        {aggregator_sys_prompt}

        Problem Examples:
        {examples}

        Solutions History: {solutions_hist}
        Evalustion History: {evalutation_hist}
        Generate your solution below:
"""
)


In [35]:
# Initializing the chain

generator_chain = LLMChain(llm=generator_llm, prompt=generator_prompt)
discriminator_chain = LLMChain(llm=discriminator_llm, prompt=disciminator_prompt)
aggregator_chain = LLMChain(llm=aggregator_llm, prompt=aggregator_prompt)
dreamstate_chain = LLMChain(llm=dreamstate_llm, prompt=dreamstate_prompt)

In [36]:
# Define Tools
tools = [
    Tool(name="Generator", func=generator_chain.run, description="Generate a solution based on the given examples and constraints."),
    Tool(name="Discriminator", func=discriminator_chain.run, description="Evaluate generated solutions for corectness and uniqueness."),
    Tool(name="Aggregator", func=aggregator_chain.run, description="Aggreagate solutions, evaluations and examples into the final solution"),
    Tool(name="Dream State", func=dreamstate_chain.run, description="Generate 10 simple and unique ideas for solving the problem"),
]

In [37]:
# Initialize Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [38]:
examples  = data_parser(all_data[0], hide_test_labels=True)[0]
ideas = []
solutions_hist = []
evalutation_hist = []
generated_solution = None

In [39]:
dream_output = dreamstate_chain.run({
    "dreamstate_sys_prompt": dreamstate_sys_prompt,
    "examples": examples,
    "solutions_hist": solutions_hist
})
ds_out = json.loads(dream_output[8:-4])
print(ds_out)

{'problem': 'Examples provided', 'ideas': [{'idea': 'Identify rows with a specific count of non-zero numbers and assign a repeating output pattern for each.', 'reason': 'Each output seems to reflect a pattern where rows with specific counts map to specific output rows. This mapping is based on seeing how many non-zero numbers exist in each row.'}, {'idea': 'Calculate the sum of each row and map it to a weighted output pattern.', 'reason': 'The sum of non-zero entries can determine the pattern in the output row, with higher sums resulting in more 5s in the output.'}, {'idea': 'Look for diagonal formations of numbers and develop an output pattern that reflects these formations.', 'reason': "Many inputs suggest diagonal alignments which could directly influence the 5's formation in the output, as seen through diagonal rows and columns."}, {'idea': 'Track the start and end indices of sequences of numbers in each row and map them to specific output sequences.', 'reason': 'Where sequences be

In [ ]:
# Parsing the Dream Output
ideas.append(ds_out)

In [27]:
ideas

[{'problem': 'Pattern recognition without examples or historical solutions provided.',
  'ideas': [{'idea': 'Linear Mapping Identification',
    'reason': 'Assume a direct linear relationship between inputs and outputs, and test this hypothesis by checking if inputs scale consistently to outputs using arithmetic operations.'},
   {'idea': 'Feature Extraction and Comparison',
    'reason': 'Identify key features or attributes in the input and correlate them to changes in the output, establishing rules based on these characteristics.'},
   {'idea': 'Pattern Increment Detection',
    'reason': 'Check if inputs and outputs follow a predictable increment or decrement pattern, such as additive or multiplicative sequences, and derive the rule from this observation.'},
   {'idea': 'Symmetrical Transformation',
    'reason': 'Look for symmetrical patterns in the input that might be mirrored in the output, suggesting transformations based on geometry or positioning.'},
   {'idea': 'Conditional L

In [26]:
ds_out

{'problem': 'Pattern recognition without examples or historical solutions provided.',
 'ideas': [{'idea': 'Linear Mapping Identification',
   'reason': 'Assume a direct linear relationship between inputs and outputs, and test this hypothesis by checking if inputs scale consistently to outputs using arithmetic operations.'},
  {'idea': 'Feature Extraction and Comparison',
   'reason': 'Identify key features or attributes in the input and correlate them to changes in the output, establishing rules based on these characteristics.'},
  {'idea': 'Pattern Increment Detection',
   'reason': 'Check if inputs and outputs follow a predictable increment or decrement pattern, such as additive or multiplicative sequences, and derive the rule from this observation.'},
  {'idea': 'Symmetrical Transformation',
   'reason': 'Look for symmetrical patterns in the input that might be mirrored in the output, suggesting transformations based on geometry or positioning.'},
  {'idea': 'Conditional Logic Rules